In [32]:
import pandas as pd
from sklearn import linear_model, preprocessing, model_selection, metrics , pipeline
import numpy as np

import matplotlib.pyplot as plt

pd.options.display.max_columns = 1000

In [14]:
path = "data_combined_cleaned.csv"
df = pd.read_csv(path)

In [15]:
df_clean = df.drop(columns=["Id"])
df_clean.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalesPrice
0,60,RL,65.0,8450,Pave,None,Reg,Lvl,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706.0,Unf,0.0,150.0,856.0,GasA,Ex,Y,SBrkr,856,854,0,1710,1.0,0.0,2,1,3,1,Gd,8,Typ,0,None,Attchd,2003.0,RFn,2.0,548.0,TA,TA,Y,0,61,0,0,0,0,None,None,None,0,2,2008,WD,Normal,208500.0
1,20,RL,80.0,9600,Pave,None,Reg,Lvl,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978.0,Unf,0.0,284.0,1262.0,GasA,Ex,Y,SBrkr,1262,0,0,1262,0.0,1.0,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2.0,460.0,TA,TA,Y,298,0,0,0,0,0,None,None,None,0,5,2007,WD,Normal,181500.0
2,60,RL,68.0,11250,Pave,None,IR1,Lvl,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486.0,Unf,0.0,434.0,920.0,GasA,Ex,Y,SBrkr,920,866,0,1786,1.0,0.0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2.0,608.0,TA,TA,Y,0,42,0,0,0,0,None,None,None,0,9,2008,WD,Normal,223500.0
3,70,RL,60.0,9550,Pave,None,IR1,Lvl,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216.0,Unf,0.0,540.0,756.0,GasA,Gd,Y,SBrkr,961,756,0,1717,1.0,0.0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3.0,642.0,TA,TA,Y,0,35,272,0,0,0,None,None,None,0,2,2006,WD,Abnorml,140000.0
4,60,RL,84.0,14260,Pave,None,IR1,Lvl,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655.0,Unf,0.0,490.0,1145.0,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1.0,0.0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3.0,836.0,TA,TA,Y,192,84,0,0,0,0,None,None,None,0,12,2008,WD,Normal,250000.0


In [16]:
df_clean = df_clean[~df.SalesPrice.isnull()]

In [17]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1460 entries, 0 to 1459
Data columns (total 79 columns):
MSSubClass       1460 non-null int64
MSZoning         1460 non-null object
LotFrontage      1460 non-null float64
LotArea          1460 non-null int64
Street           1460 non-null object
Alley            1460 non-null object
LotShape         1460 non-null object
LandContour      1460 non-null object
LotConfig        1460 non-null object
LandSlope        1460 non-null object
Neighborhood     1460 non-null object
Condition1       1460 non-null object
Condition2       1460 non-null object
BldgType         1460 non-null object
HouseStyle       1460 non-null object
OverallQual      1460 non-null int64
OverallCond      1460 non-null int64
YearBuilt        1460 non-null int64
YearRemodAdd     1460 non-null int64
RoofStyle        1460 non-null object
RoofMatl         1460 non-null object
Exterior1st      1460 non-null object
Exterior2nd      1460 non-null object
MasVnrType       1460 no

In [18]:
df_clean.describe().T

,count,mean,std,min,25%,50%,75%,max
MSSubClass,1460.0,56.897260,42.300571,20.0,20.00,50.0,70.00,190.0
LotFrontage,1460.0,70.176370,22.433457,21.0,60.00,70.0,80.00,313.0
LotArea,1460.0,10516.828082,9981.264932,1300.0,7553.50,9478.5,11601.50,215245.0
OverallQual,1460.0,6.099315,1.382997,1.0,5.00,6.0,7.00,10.0
OverallCond,1460.0,5.575342,1.112799,1.0,5.00,5.0,6.00,9.0
YearBuilt,1460.0,1971.267808,30.202904,1872.0,1954.00,1973.0,2000.00,2010.0
YearRemodAdd,1460.0,1984.865753,20.645407,1950.0,1967.00,1994.0,2004.00,2010.0
MasVnrArea,1460.0,103.117123,180.731373,0.0,0.00,0.0,164.25,1600.0
BsmtFinSF1,1460.0,443.639726,456.098091,0.0,0.00,383.5,712.25,5644.0
BsmtFinSF2,1460.0,46.549315,161.319273,0.0,0.00,0.0,0.00,1474.0


In [29]:
df = df_clean

In [33]:
target = "SalesPrice"
X = df.copy()
del X[target]
X = pd.get_dummies(X, drop_first=True)
y = df[target]

X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size = 0.3, random_state = 123)

est = pipeline.Pipeline([
    ("poly", preprocessing.PolynomialFeatures(degree=2
                                    , include_bias=False)),
    ("scaler", preprocessing.StandardScaler()),
    ("lr", linear_model.LinearRegression())
])

#est = linear_model.LinearRegression()
est.fit(X_train, y_train)

y_train_pred = est.predict(X_train)
y_test_pred = est.predict(X_test)

r2_train = metrics.r2_score(y_train, y_train_pred)
r2_test = metrics.r2_score(y_test, y_test_pred)
rmse_train = np.sqrt(metrics.mean_squared_error(y_train, y_train_pred))
rmse_test = np.sqrt(metrics.mean_squared_error(y_test, y_test_pred))

print("R2_train: %f, \nR2_test: %f\nrmse train: %f\nrmse_test: %f" 
      % (r2_train, r2_test, rmse_train, rmse_test))

R2_train: 1.000000, 
R2_test: 0.838407
rmse train: 0.000000
rmse_test: 30451.497318


In [52]:
target = "SalesPrice"
X = df.copy()
del X[target]
X = pd.get_dummies(X, drop_first=True)
y = df[target]

X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y
                            , test_size = 0.3, random_state = 123)

est = pipeline.Pipeline([
    ("poly", preprocessing.PolynomialFeatures(degree=1
                                    , include_bias=False)),
    ("scaler", preprocessing.StandardScaler()),
    ("lr", linear_model.Lasso(alpha = 400.0, max_iter=1000)) #alpha is the regularization param
])

#est = linear_model.LinearRegression()
est.fit(X_train, y_train)

y_train_pred = est.predict(X_train)
y_test_pred = est.predict(X_test)

r2_train = metrics.r2_score(y_train, y_train_pred)
r2_test = metrics.r2_score(y_test, y_test_pred)
rmse_train = np.sqrt(metrics.mean_squared_error(y_train, y_train_pred))
rmse_test = np.sqrt(metrics.mean_squared_error(y_test, y_test_pred))

print("R2_train: %f, \nR2_test: %f\nrmse train: %f\nrmse_test: %f" 
      % (r2_train, r2_test, rmse_train, rmse_test))

R2_train: 0.925663, 
R2_test: 0.852360
rmse train: 22061.896574
rmse_test: 29107.137699


In [51]:
est.steps[-1][-1].coef_

array([-4.34192279e+03,  1.47153308e+02,  5.28906733e+03,  9.51182600e+03,
        6.44972477e+03,  9.02814545e+03,  1.83320970e+03,  2.15015575e+03,
        8.68385032e+03,  2.04729458e+03, -0.00000000e+00,  6.44390052e+03,
        0.00000000e+00,  2.14997164e+03, -2.55256187e+03,  2.75473114e+04,
        8.54680147e+02, -0.00000000e+00,  8.47238732e+02,  0.00000000e+00,
       -3.35761342e+03, -1.33889927e+03,  3.70709480e+03,  3.32734252e+03,
       -4.88124683e+02,  5.51243401e+03,  1.29106284e+03,  1.99762218e+03,
        2.00072615e+02, -1.03086199e+02,  7.58458891e+02,  1.77515447e+03,
        9.11286169e+04,  0.00000000e+00, -4.07446157e+02, -1.12922639e+02,
        2.70422144e+02,  5.96776053e+02,  4.44457040e+03,  2.71723248e+03,
        3.65210477e+03,  0.00000000e+00, -1.10440173e+02,  1.81131068e+03,
        1.41457668e+01,  1.31624490e+03,  2.74108865e+03, -6.63770157e+02,
        2.46167850e+03,  8.13085253e+02, -9.61612523e+02, -5.99843866e+02,
       -6.12428877e+02,  

In [53]:
target  ==  "SalesPrice"
X = df.copy()
del X[target]
X = pd.get_dummies(X, drop_first=True)
y = df[target]

X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y
                            , test_size = 0.3, random_state = 123)

est = pipeline.Pipeline([
    ("poly", preprocessing.PolynomialFeatures(degree=1
                                    , include_bias=False)),
    ("scaler", preprocessing.StandardScaler()),
    ("lr", linear_model.Lasso(alpha = 400.0, max_iter=2000)) #alpha is the regularization param
])

#est = linear_model.LinearRegression()
est.fit(X_train, y_train)

y_train_pred = est.predict(X_train)
y_test_pred = est.predict(X_test)

r2_train = metrics.r2_score(y_train, y_train_pred)
r2_test = metrics.r2_score(y_test, y_test_pred)
rmse_train = np.sqrt(metrics.mean_squared_error(y_train, y_train_pred))
rmse_test = np.sqrt(metrics.mean_squared_error(y_test, y_test_pred))

print("R2_train: %f, \nR2_test: %f\nrmse train: %f\nrmse_test: %f" 
      % (r2_train, r2_test, rmse_train, rmse_test))

R2_train: 0.925663, 
R2_test: 0.852360
rmse train: 22061.896574
rmse_test: 29107.137699


In [54]:
est.steps[-1][-1].coef_

array([-5.41932298e+03, -0.00000000e+00,  4.11949986e+03,  1.20465235e+04,
        6.05166848e+03,  7.35203337e+03,  1.57727364e+03,  1.70420875e+03,
        6.47153208e+03,  7.88405030e+02, -0.00000000e+00,  3.81636677e+03,
        0.00000000e+00,  0.00000000e+00, -2.48793805e+03,  2.69773285e+04,
        1.50377961e+03, -0.00000000e+00,  1.14312301e+03,  0.00000000e+00,
       -2.12641683e+03, -1.30745778e+03,  3.26683376e+03,  2.35253407e+03,
        0.00000000e+00,  7.08060860e+03,  0.00000000e+00,  2.27045935e+03,
        2.47640420e+02, -6.04361922e+02,  3.21050130e+02,  1.12824533e+03,
        4.57774417e+04, -0.00000000e+00, -2.14473566e+02, -0.00000000e+00,
       -0.00000000e+00,  0.00000000e+00,  1.27867700e+03, -0.00000000e+00,
        2.69776213e+03,  0.00000000e+00, -0.00000000e+00,  1.65947408e+03,
       -6.09714211e+02,  2.65710796e+02,  2.70141231e+03, -0.00000000e+00,
        2.27941024e+03,  1.20328672e+03, -3.05462183e+02, -4.21689688e+02,
       -0.00000000e+00,  

In [55]:
target = "SalesPrice"
X = df.copy()
del X[target]
X = pd.get_dummies(X, drop_first=True)
y = df[target]

X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y
                            , test_size = 0.3, random_state = 123)

est = pipeline.Pipeline([
    ("poly", preprocessing.PolynomialFeatures(degree=1
                                    , include_bias=False)),
    ("scaler", preprocessing.StandardScaler()),
    ("lr", linear_model.Ridge(alpha = 400.0, max_iter=2000)) #alpha is the regularization param
])

#est = linear_model.LinearRegression()
est.fit(X_train, y_train)

y_train_pred = est.predict(X_train)
y_test_pred = est.predict(X_test)

r2_train = metrics.r2_score(y_train, y_train_pred)
r2_test = metrics.r2_score(y_test, y_test_pred)
rmse_train = np.sqrt(metrics.mean_squared_error(y_train, y_train_pred))
rmse_test = np.sqrt(metrics.mean_squared_error(y_test, y_test_pred))

print("R2_train: %f, \nR2_test: %f\nrmse train: %f\nrmse_test: %f" 
      % (r2_train, r2_test, rmse_train, rmse_test))

R2_train: 0.896578, 
R2_test: 0.864171
rmse train: 26022.351097
rmse_test: 27918.633972


In [56]:
est.steps[-1][-1].coef_

array([-2.96465544e+03,  3.94840115e+02,  3.29267454e+03,  8.47188772e+03,
        3.28025892e+03,  2.07580358e+03,  2.59693944e+03,  3.27645033e+03,
        3.62282230e+03,  9.02790620e+02, -1.02596927e+01,  4.06316523e+03,
        5.84985850e+03,  4.85375966e+03, -7.34181389e+02,  8.18336890e+03,
        1.92374228e+03, -6.41398741e+01,  3.96093089e+03,  1.96154494e+03,
       -2.81470403e+02, -1.74057613e+03,  5.15498403e+03,  3.31980313e+03,
        5.68743199e+02,  5.12593661e+03,  3.28094854e+03,  2.71921806e+03,
        1.09428287e+03, -4.19291035e+02,  9.22652738e+02,  1.30899108e+03,
        3.63368606e+03,  2.27515446e+02, -1.04308236e+02, -2.05049106e+02,
        4.44100240e+02,  1.72329226e+02,  1.68000059e+03, -6.15791584e+02,
        2.31672917e+03,  2.40965737e+02,  1.67902747e+02,  1.62029188e+03,
       -1.89229653e+03, -3.15908864e+02,  2.58114247e+03, -3.51840087e+01,
        1.33577432e+03,  1.48647801e+03, -8.94772474e+02, -7.42429778e+02,
       -4.58641753e+02,  

In [41]:
param_grid = {
    "lr__alpha": 100 * np.linspace(10,12, 20)
}
gs = model_selection.GridSearchCV(est, param_grid, cv = 5, verbose = True)
gs.fit(X_train, y_train)

y_train_pred = gs.predict(X_train)
y_test_pred = gs.predict(X_test)

r2_train = metrics.r2_score(y_train, y_train_pred)
r2_test = metrics.r2_score(y_test, y_test_pred)
rmse_train = np.sqrt(metrics.mean_squared_error(y_train, y_train_pred))
rmse_test = np.sqrt(metrics.mean_squared_error(y_test, y_test_pred))

print("R2_train: %f, \nR2_test: %f\nrmse train: %f\nrmse_test: %f" 
      % (r2_train, r2_test, rmse_train, rmse_test))

Fitting 5 folds for each of 20 candidates, totalling 100 fits
R2_train: 0.860159, 
R2_test: 0.849497
rmse train: 30259.164013
rmse_test: 29387.985154


[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    4.6s finished


In [42]:
gs.best_params_

{'lr__alpha': 1105.2631578947369}

In [43]:
10 * np.linspace(10,100, 20)

array([ 100.        ,  147.36842105,  194.73684211,  242.10526316,
        289.47368421,  336.84210526,  384.21052632,  431.57894737,
        478.94736842,  526.31578947,  573.68421053,  621.05263158,
        668.42105263,  715.78947368,  763.15789474,  810.52631579,
        857.89473684,  905.26315789,  952.63157895, 1000.        ])

In [44]:

import  pickle

In [45]:
with open("model.pickle", "wb") as f:
    pickle.dump(gs, f)

In [46]:
with open("model.pickle", "rb") as f:
    model_reloaded = pickle.load(f)
    
y_train_pred = model_reloaded.predict(X_train)
y_test_pred = model_reloaded.predict(X_test)

r2_train = metrics.r2_score(y_train, y_train_pred)
r2_test = metrics.r2_score(y_test, y_test_pred)
rmse_train = np.sqrt(metrics.mean_squared_error(y_train, y_train_pred))
rmse_test = np.sqrt(metrics.mean_squared_error(y_test, y_test_pred))

print("R2_train: %f, \nR2_test: %f\nrmse train: %f\nrmse_test: %f" 
      % (r2_train, r2_test, rmse_train, rmse_test))

R2_train: 0.860159, 
R2_test: 0.849497
rmse train: 30259.164013
rmse_test: 29387.985154


In [47]:
!ls -lh

'ls' is not recognized as an internal or external command,
operable program or batch file.


In [69]:
target = "SalesPrice"
X = df.copy()
del X[target]
X = pd.get_dummies(X, drop_first=True)
y = df[target]

X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y
                            , test_size = 0.3, random_state = 123)

est = pipeline.Pipeline([
    ("poly", preprocessing.PolynomialFeatures(degree=1
                                    , include_bias=False)),
    ("scaler", preprocessing.StandardScaler()),
    ("lr", linear_model.ElasticNet(alpha = 1.63157895, max_iter=500)) #alpha is the regularization param
])

#est = linear_model.LinearRegression()
est.fit(X_train, y_train)

y_train_pred = est.predict(X_train)
y_test_pred = est.predict(X_test)

r2_train = metrics.r2_score(y_train, y_train_pred)
r2_test = metrics.r2_score(y_test, y_test_pred)
rmse_train = np.sqrt(metrics.mean_squared_error(y_train, y_train_pred))
rmse_test = np.sqrt(metrics.mean_squared_error(y_test, y_test_pred))

print("R2_train: %f, \nR2_test: %f\nrmse train: %f\nrmse_test: %f" 
      % (r2_train, r2_test, rmse_train, rmse_test))

R2_train: 0.872575, 
R2_test: 0.856143
rmse train: 28884.653747
rmse_test: 28731.804876


In [66]:
param_grid = {
    "lr__alpha": 1 * np.linspace(1,5, 20)
}
gs = model_selection.GridSearchCV(est, param_grid, cv = 5, verbose = True)
gs.fit(X_train, y_train)

y_train_pred = gs.predict(X_train)
y_test_pred = gs.predict(X_test)

r2_train = metrics.r2_score(y_train, y_train_pred)
r2_test = metrics.r2_score(y_test, y_test_pred)
rmse_train = np.sqrt(metrics.mean_squared_error(y_train, y_train_pred))
rmse_test = np.sqrt(metrics.mean_squared_error(y_test, y_test_pred))

print("R2_train: %f, \nR2_test: %f\nrmse train: %f\nrmse_test: %f" 
      % (r2_train, r2_test, rmse_train, rmse_test))

Fitting 5 folds for each of 20 candidates, totalling 100 fits
R2_train: 0.849153, 
R2_test: 0.842643
rmse train: 31427.440044
rmse_test: 30049.751639


[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    5.7s finished


In [71]:
gs.best_params_

{'lr__alpha': 2.6842105263157894}

In [72]:
gs


GridSearchCV(cv=5, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('poly', PolynomialFeatures(degree=1, include_bias=False, interaction_only=False)), ('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('lr', ElasticNet(alpha=2.0, copy_X=True, fit_intercept=True, l1_ratio=0.5,
      max_iter=500, normalize=False, positive=False, precompute=False,
      random_state=None, selection='cyclic', tol=0.0001, warm_start=False))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'lr__alpha': array([1.     , 1.21053, 1.42105, 1.63158, 1.84211, 2.05263, 2.26316,
       2.47368, 2.68421, 2.89474, 3.10526, 3.31579, 3.52632, 3.73684,
       3.94737, 4.15789, 4.36842, 4.57895, 4.78947, 5.     ])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=True)